In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-0.530005,-0.11573,-0.170989,0.33106,1.427702,1.375176,0.815358,-0.687604,0.41171,0.599486
0,-0.530294,-0.626899,-0.607625,1.090091,1.167637,2.505301,-0.291641,-0.093724,-0.578608,-1.080053
1,-2.369443,-2.211986,0.806646,2.62867,-0.073144,0.864496,-1.666375,-0.317636,-0.003078,-2.478897
0,-0.413598,-0.622732,-1.40097,1.875616,1.846896,0.332353,0.108449,1.438836,0.868123,-0.978085
1,-3.330914,-2.785849,2.447746,1.757732,-0.869147,2.988697,-1.752147,-2.838499,-0.599155,-1.712332


## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
scaled = t.scale(df, cols=features, strategy="standard")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.416436,0.812041,-0.314945,-0.50577,0.556433,0.302315,0.939184,-0.114853,0.056324,0.683929
0,0.416243,0.343898,-0.615854,0.036933,0.400233,1.21756,-0.034063,0.23666,-0.58874,-0.356561
1,-0.814165,-1.107766,0.358796,1.137009,-0.345002,-0.111266,-1.242693,0.104128,-0.213857,-1.223157
0,0.494314,0.347716,-1.16259,0.598579,0.808208,-0.542228,0.317687,1.143774,0.353618,-0.293391
1,-1.457398,-1.633325,1.489765,0.514293,-0.823096,1.609045,-1.318103,-1.387956,-0.602124,-0.748263


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,target
0,0.416439,0.812045,-0.314946,-0.505772,0.556436,0.302317,0.939188,-0.114854,0.056324,0.683932,0
1,0.416245,0.343900,-0.615857,0.036934,0.400235,1.217567,-0.034063,0.236661,-0.588743,-0.356563,0
2,-0.814169,-1.107772,0.358798,1.137015,-0.345004,-0.111266,-1.242700,0.104129,-0.213858,-1.223163,1
3,0.494317,0.347717,-1.162596,0.598582,0.808212,-0.542231,0.317688,1.143779,0.353619,-0.293392,0
4,-1.457405,-1.633333,1.489772,0.514295,-0.823100,1.609053,-1.318109,-1.387963,-0.602127,-0.748267,1


In [9]:
%%timeit
scaled = t.scale(df, cols=features, strategy="standard")

2.61 ms ± 99.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


12.2 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import dsds.transform as t
t.impute(df, cols=features, strategy="median").head(3) 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-0.530005,-0.11573,-0.170989,0.33106,1.427702,1.375176,0.815358,-0.687604,0.41171,0.599486
0,-0.530294,-0.626899,-0.607625,1.090091,1.167637,2.505301,-0.291641,-0.093724,-0.578608,-1.080053
1,-2.369443,-2.211986,0.806646,2.62867,-0.073144,0.864496,-1.666375,-0.317636,-0.003078,-2.478897


In [12]:
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head(3)

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0.0,-0.530005,-0.115730,-0.170989,0.331060,1.427702,1.375176,0.815358,-0.687604,0.411710,0.599486
1,0.0,-0.530294,-0.626899,-0.607625,1.090091,1.167637,2.505301,-0.291641,-0.093724,-0.578608,-1.080053
2,1.0,-2.369443,-2.211986,0.806646,2.628670,-0.073144,0.864496,-1.666375,-0.317636,-0.003078,-2.478897


In [13]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)


117 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
t.impute(df, cols=features, strategy="median")

1.26 ms ± 46.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Fscore

In [15]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [16]:
fs._f_score(df, target=target, num_list = features)

array([4.17004737e+03, 1.94417782e-01, 1.97369637e+04, 8.76451966e+01,
       1.00225550e+04, 1.00965762e-01, 1.68644247e+04, 9.67350729e+03,
       2.26720328e+04, 1.06798295e+04])

In [17]:
# The more core, the bigger the difference. Data here is not big enough to show the difference
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.01s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",4170.047367,0.0
"""column_1""",0.194418,0.659266
"""column_2""",19736.963691,0.0
"""column_3""",87.645197,7.9867e-21
"""column_4""",10022.555007,0.0
"""column_5""",0.100966,0.750674
"""column_6""",16864.424736,0.0
"""column_7""",9673.50729,0.0
"""column_8""",22672.032803,0.0


In [18]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


,feature,f_value,p_value
0,column_0,4170.047367,0.000000e+00
1,column_1,0.194418,6.592658e-01
2,column_2,19736.963691,0.000000e+00
3,column_3,87.645197,7.986694e-21
4,column_4,10022.555007,0.000000e+00
5,column_5,0.100966,7.506744e-01
6,column_6,16864.424736,0.000000e+00
7,column_7,9673.507290,0.000000e+00
8,column_8,22672.032803,0.000000e+00
9,column_9,10679.829541,0.000000e+00


In [19]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.01s in computing Fscore.


,feature,f_value,p_value
0,column_0,4170.047367,0.000000e+00
1,column_1,0.194418,6.592658e-01
2,column_2,19736.963691,0.000000e+00
3,column_3,87.645197,7.986694e-21
4,column_4,10022.555007,0.000000e+00
5,column_5,0.100966,7.506744e-01
6,column_6,16864.424736,0.000000e+00
7,column_7,9673.507290,0.000000e+00
8,column_8,22672.032803,0.000000e+00
9,column_9,10679.829541,0.000000e+00


# Mutual Information Score

In [20]:
# Vastly faster than sklearn. Finished in 0.7s in this run
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 10/10 [00:00<00:00, 12.73it/s]


feature,estimated_mi
str,f64
"""column_6""",0.121489
"""column_8""",0.113399
"""column_2""",0.103288
"""column_4""",0.061133
"""column_7""",0.058957
"""column_9""",0.053517
"""column_0""",0.029735
"""column_1""",0.025451
"""column_3""",0.011829


In [21]:
# A wrapper for more apples to apples comparison
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [22]:
# The reason sklearn's impl is slow is that it did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality. Finished in 4.4s in this run
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_6""",0.121489
"""column_8""",0.113399
"""column_2""",0.103288
"""column_4""",0.061133
"""column_7""",0.058957
"""column_9""",0.053517
"""column_0""",0.029735
"""column_1""",0.025451
"""column_3""",0.011823


# MRMR Feature selection Strategy

In [23]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [24]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [25]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 10/10 [00:06<00:00,  1.60it/s]

Spent 9.17s to compute mrmr.


['column_8',
 'column_0',
 'column_6',
 'column_2',
 'column_4',
 'column_9',
 'column_7',
 'column_3',
 'column_5',
 'column_1']

In [26]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 1428.43it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_8',
 'column_0',
 'column_6',
 'column_2',
 'column_4',
 'column_9',
 'column_7',
 'column_3',
 'column_5',
 'column_1']

In [27]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 555.45it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_8',
 'column_0',
 'column_6',
 'column_2',
 'column_4',
 'column_9',
 'column_7',
 'column_3',
 'column_5',
 'column_1']

# Power Transform

In [28]:
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Spent 0.22s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-0.529729,-0.115949,-0.170658,0.339347,1.44585,1.380586,0.869039,-0.692359,0.407772,0.607435
0,-0.530018,-0.632517,-0.60392,1.167478,1.180363,2.520376,-0.284258,-0.093827,-0.586173,-1.057333
1,-2.365503,-2.264776,0.812939,2.993101,-0.073079,0.866859,-1.495149,-0.318748,-0.003078,-2.384629
0,-0.413425,-0.62828,-1.384313,2.079178,1.875244,0.33275,0.109552,1.421166,0.8525,-0.959043
1,-3.324018,-2.86349,2.491719,1.939412,-0.861743,3.008867,-1.565914,-2.89546,-0.607229,-1.661654


In [29]:
from sklearn.preprocessing import power_transform

In [30]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 0.72s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0,-0.529729,-0.115949,-0.170658,0.339347,1.445850,1.380586,0.869039,-0.692359,0.407772,0.607435
1,0,-0.530018,-0.632517,-0.603920,1.167478,1.180363,2.520376,-0.284258,-0.093827,-0.586173,-1.057333
2,1,-2.365503,-2.264776,0.812939,2.993101,-0.073079,0.866859,-1.495149,-0.318748,-0.003078,-2.384629
3,0,-0.413425,-0.628280,-1.384313,2.079178,1.875244,0.332750,0.109552,1.421166,0.852500,-0.959043
4,1,-3.324018,-2.863490,2.491719,1.939412,-0.861743,3.008867,-1.565914,-2.895460,-0.607229,-1.661654


# GC

In [31]:
import gc 
gc.collect()

243